In [19]:
from datetime import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import UFC_Scrape_Library as LIB

In [2]:
urls_yearly_events = []
currentyear = datetime.now().year

for year in range(1994, currentyear + 1):
    newurl = 'https://mmadecisions.com/decisions-by-event/' + str(year) + '/'
    urls_yearly_events.append(newurl)

urls_yearly_events

['https://mmadecisions.com/decisions-by-event/1994/',
 'https://mmadecisions.com/decisions-by-event/1995/',
 'https://mmadecisions.com/decisions-by-event/1996/',
 'https://mmadecisions.com/decisions-by-event/1997/',
 'https://mmadecisions.com/decisions-by-event/1998/',
 'https://mmadecisions.com/decisions-by-event/1999/',
 'https://mmadecisions.com/decisions-by-event/2000/',
 'https://mmadecisions.com/decisions-by-event/2001/',
 'https://mmadecisions.com/decisions-by-event/2002/',
 'https://mmadecisions.com/decisions-by-event/2003/',
 'https://mmadecisions.com/decisions-by-event/2004/',
 'https://mmadecisions.com/decisions-by-event/2005/',
 'https://mmadecisions.com/decisions-by-event/2006/',
 'https://mmadecisions.com/decisions-by-event/2007/',
 'https://mmadecisions.com/decisions-by-event/2008/',
 'https://mmadecisions.com/decisions-by-event/2009/',
 'https://mmadecisions.com/decisions-by-event/2010/',
 'https://mmadecisions.com/decisions-by-event/2011/',
 'https://mmadecisions.com/d

In [ ]:
df_event = pd.read_csv('MMA_Events.csv')
urls_events = df_event['url'].tolist()
    
for url in urls_yearly_events:

    if url in urls_events:
        continue

    df_event_new = LIB.scrape_event_data(url)

    df_event_all = pd.concat([df_event, df_event_new], ignore_index=True)

# Sort descending by date
df_event_all['Date'] = pd.to_datetime(df_event_all['Date']).dt.date
df_event_all = df_event_all.sort_values(by='Date', ascending=False)

df_event_all.head(10)

,Date,Event,NumFights,url
1514,2025-05-31,UFC on ESPN 68: Blanchfield vs. Barber,5,https://mmadecisions.com/event/1549/UFC-on-ESP...
1,2025-04-26,UFC on ESPN 66: Machado Garry vs. Prates,4,https://mmadecisions.com/event/1543/UFC-on-ESP...
0,2025-04-26,CW 188: Cage Warriors 188,8,https://mmadecisions.com/event/1542/CW-188-Cag...
2,2025-04-19,CW 187: Glasgow,3,https://mmadecisions.com/event/1541/CW-187-Gla...
3,2025-04-18,PFL 3: 2025 Season,2,https://mmadecisions.com/event/1540/PFL-3-2025...
4,2025-04-12,UFC 314: Volkanovski vs. Lopes,5,https://mmadecisions.com/event/1538/UFC-314-Vo...
5,2025-04-11,PFL 2: 2025 Season,3,https://mmadecisions.com/event/1539/PFL-2-2025...
6,2025-04-05,UFC on ESPN 65: Emmett vs. Murphy,8,https://mmadecisions.com/event/1536/UFC-on-ESP...
7,2025-04-03,PFL 1: 2025 Season,2,https://mmadecisions.com/event/1537/PFL-1-2025...
8,2025-03-29,UFC on ESPN 64: Moreno vs. Erceg,7,https://mmadecisions.com/event/1535/UFC-on-ESP...


In [ ]:
# Save to MMA_Events csv
df_event_all.to_csv('MMA_Events.csv', index=False)

In [20]:
all_fight_data = []

df_decision = pd.read_csv('MMA_Decisions.csv')
urls_decisions = df_decision['url'].tolist()

for url in df_decision['url']:

  if url in urls_events:
      continue
  
  df_decision_new = LIB.scrape_fight_data(url)

  df_decision = pd.concat([df_decision, df_decision_new], ignore_index=True)
    
# Sort descending by date
df_decision = df_decision.sort_values(by='url', ascending=False)

df_decision_new.head(10)


NameError: name 'all_fight_data' is not defined

In [ ]:
# Save to MMA_Events csv
df_decision_all.to_csv('MMA_Decisions.csv', index=False)

In [ ]:
list_scorecard = []
error_urls = []
urls_scorecard = pd.read_csv('MMA_Scorecards.csv')['url'].tolist()

for url in df_fights['url']:
    #Check if the scorecard has already been scraped
    if url in urls_scorecard:
        continue

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    tables = soup.find_all('table')
    try:
        second_table = tables[2]
    except IndexError:
        try:
            second_table = tables[1]
        except IndexError:
            error_urls.append(url)
            continue

    # Find all tr elements with class "top-row" or "decision"
    rows = second_table.find_all('tr', class_=['top-row', 'decision'])

    # Filter out rows with class "decision-bottom2" and "Tale of the Tape" text
    filtered_rows = [
        row for row in rows
        if 'decision-bottom2' not in row.get('class', []) and "TALE OF THE TAPE" not in row.text
    ]

    # Initialize an empty list to store data
    data = []

    # Extract data and append to the list
    for row in filtered_rows:
        row_text = row.get_text(strip=False)  # Get the text content of the row
        row_data = row_text.splitlines()  # Split the text into lines
        data.append(row_data)

    # Create the DataFrame
    df_scorecards = pd.DataFrame(data)
    df_scorecards = df_scorecards[[1, 2, 3]]
    df_scorecards = df_scorecards[~df_scorecards[1].str.contains("LEGEND")]

    # Reset the index (optional)
    df_scorecards = df_scorecards.reset_index(drop=True)
    
    fight = url.rsplit('/', 1)[-1].replace('-', ' ').strip()
    fighter_one = fight.rsplit('vs', 1)[0].strip()
    try:
        fighter_two = fight.rsplit('vs', 1)[1].strip()
    except IndexError:
        error_urls.append(url)
        continue

    # Find the index of the row with blank values in columns 1 and 3
    try:
        blank_row_index = df_scorecards[(df_scorecards[1] == '') & (df_scorecards[3] == '')].index[0]
    except IndexError:
        error_urls.append(url)
        continue

    # Remove all rows above the blank row
    df_scorecards = df_scorecards[blank_row_index:]
    
    # Reset the index
    df_scorecards = df_scorecards.reset_index(drop=True)

    df_scorecards = df_scorecards[~df_scorecards[1].str.contains("ROUND")]

    df_scorecards['Judge'] = ''

    current_judge = None

    for index, row in df_scorecards.iterrows():
        # Extract the value from column 2
        value = row[2]

        # Check if the value contains a judge's name
        if any(char.isalpha() for char in value):  # Check if any character is alphabetic
            current_judge = value  # Update current_judge if it's a judge's name

        # Assign the current_judge to the 'Judge' column
        df_scorecards.loc[index, 'Judge'] = current_judge
    
    df_scorecards = df_scorecards[df_scorecards[1] != '']
    df_scorecards = df_scorecards.rename(columns={1: 'Round', 2: 'ScoreOne', 3: 'ScoreTwo'})

    df_scorecards['FighterOne'] = fighter_one
    df_scorecards['FighterTwo'] = fighter_two
    df_scorecards['Fight'] = fight
    df_scorecards['url'] = url

    df_scorecards = df_scorecards[['Fight', 'Judge', 'Round', 'FighterOne', 'FighterTwo', 'ScoreOne', 'ScoreTwo', 'url']]
    
    list_scorecard.append(df_scorecards)

if len(list_scorecard) > 0:
    df_scorecards = pd.concat(list_scorecard, ignore_index=True)

print("Errors:",len(error_urls))

df_scorecards.head(10)

ValueError: No objects to concatenate

In [ ]:
# Sort descending by date
df_scorecards = df_scorecards.sort_values(by='url', ascending=False)

# Save to MMA_Scorecards csv
df_scorecards.to_csv('MMA_Scorecards.csv', index=False)

In [ ]:
pd.read_csv('MMA_Scorecards.csv')['url'].tolist()

63888
